In [1]:
import numpy as np
import pandas as pd

import random
random.seed(42)
np.random.seed(42)

import warnings
warnings.filterwarnings("ignore")


In [3]:
df = pd.read_csv(
    r"C:\Users\HP\Text-Based-Risk-Classification-System\Text-Based-Risk-Classification-System\data\spam.csv",
    encoding="latin-1"
)


df = df[['v1', 'v2']]
df.columns = ['label', 'text']

df['label'] = df['label'].map({'ham': 0, 'spam': 1})

df = df.drop_duplicates().reset_index(drop=True)


In [4]:
# basic length features
df['message_length'] = df['text'].str.len()
df['word_count'] = df['text'].str.split().str.len()

# numeric-related features
df['number_count'] = df['text'].str.count(r'\d+')
df['contains_number'] = (df['number_count'] > 0).astype(int)

# special characters
df['special_char_count'] = df['text'].str.count(r'[^\w\s]')

# url indicator
df['has_url'] = df['text'].str.contains(r'http|www', regex=True).astype(int)


In [5]:
df['text_model'] = df['text'].str.lower()
